Thoth Gunter july 2016
# Updating Based analysis

Originally the basic cuts analysis used the cuts below.  After reviewing ATLAS' WW cross-section analysis the an additional cut was made for same flavor events METProj > 45.  

Original cuts:
- mll > 12
- qT > 30
- MET Projected > 20
- number of extra leptons  = 0
- number of jet < 0, 1

In addition: 
if lepton pair is the same flavor: 
- mll < 76 or mll > 106 
- qT > 45 

In [1]:
import root_pandas as rp
import numpy as np

In [2]:
df = rp.read_root("pan_process.root")
df.head()

,sub_lep_eta,runNum,nBJet,lep_Charge,HT,WW,DY,numb_jets,lead_lep_eta,Da,...,ZZ,npv,dPhiLLMET,lep_Type,METProj_parr,lep3_pt,mll,rf_result,weights,lead_lep_type
0,1.202665,1,0,-1,0.000000,1,0,0,-1.430588,0,...,0,19,2.813456,1,216.957748,0,170.901596,0.999333,1.302513,13
1,-1.885126,1,0,-1,57.011757,1,0,1,-1.709151,0,...,0,28,0.349567,1,130.962906,0,83.166290,0.919333,0.550025,13
2,1.828307,1,0,-1,189.272278,1,0,1,1.029058,0,...,0,34,1.225897,1,843.469299,0,70.566330,0.999333,0.236836,13
3,1.337142,1,0,-1,227.126526,1,0,2,-0.363505,0,...,0,17,0.706413,1,7163.877930,0,154.645264,0.884667,1.946137,13
4,0.653018,1,0,-1,37.855759,1,0,1,2.135821,0,...,0,20,3.021460,-1,14.918805,0,162.724945,0.876000,1.123370,13


### Implementing cuts

In [ ]:
initial_cuts = (df.mll > 12 ) & (df.qT > 30) & (df.METProj > 20) & (df.numbExtraLep == 0)
same_flavor_cut = (df.lep_Type < 0)
dif_flavor_cut = (df.lep_Type > 0)
z_peak_cut = (df.mll > 106) | (df.mll < 76)
met_proj_cut = (df.METProj > 45)


In [ ]:
print "Same flavor cuts"
basic_sf_0j_cuts = initial_cuts & (df.qT > 45) &same_flavor_cut & met_proj_cut & z_peak_cut& (df.numb_jets <= 1)


In [ ]:
print "Different flavor cuts"
basic_df_0j_cuts = initial_cuts & dif_flavor_cut &(df.numb_jets <= 1)


In [ ]:
# uncertainty on process
unc_mc_process = { "WW": .05, "DY": .03, "TT": 0.05, "ZZ": 0.1, "WZ":.1 }
# process ratio scalings
scales = {"WW": 1.33 * 19.4e3 * 59.8 /  6066954.0 ,\
          "DY": 19.4e3 * 3531.9 / 30459500 ,\
          "TT": 19.4e3 * 25.81 / 12011428,\
          "ZZ": 19.4e3 * 9.03 / 200000,\
          "WZ": 19.4e3 * 22.45 / 400000, "WJ": 1 }

In [ ]:
results_0j = { "same_flavor":{"WW": df[ basic_sf_0j_cuts &(df.WW == 1)].shape[0] * scales["WW"] ,
            "DY": df[ basic_sf_0j_cuts &(df.DY == 1)].shape[0] * scales["DY"],
            "TT": df[ basic_sf_0j_cuts &(df.TT == 1)].shape[0] * scales["TT"] ,
            "ZZ": df[ basic_sf_0j_cuts &(df.ZZ == 1)].shape[0] * scales["ZZ"],
            "WZ": df[ basic_sf_0j_cuts &(df.WZ == 1)].shape[0] * scales["WZ"] }}

results_0j["diff_flavor"] = {"WW": df[ basic_df_0j_cuts &( df.WW == 1)].shape[0] * scales["WW"] ,
                            "DY": df[ basic_df_0j_cuts &( df.DY == 1)].shape[0] * scales["DY"] ,
                            "TT": df[ basic_df_0j_cuts &(df.TT == 1)].shape[0] * scales["TT"] ,
                            "ZZ": df[ basic_df_0j_cuts &(df.ZZ == 1)].shape[0] * scales["ZZ"] ,
                            "WZ": df[ basic_df_0j_cuts &(df.WZ == 1)].shape[0] * scales["WZ"] }

In [ ]:
def create_table( data, round_digit ):
    for flavor in data.keys():
        print flavor
        print "\t",{ process : round(data[flavor][process], round_digit) for process in data[flavor].keys() }

In [ ]:
# results
create_table(results_0j, 0)

In [ ]:
def combine_unc( data ):
    comb_unc = {}
    for process in data[data.keys()[0]].keys():
        comb_unc[process] = 0
    for flavor in data.keys():
        for process in data[flavor].keys():
            comb_unc[process] += pow( data[flavor][process], 2 )
    print { process : round( pow( comb_unc[process], .5) , 2) for process in comb_unc.keys()}

## Uncertainty procedure 


### Statistical uncertainty of a process is defined as: 

$ \large {{scaling * \sqrt{N_{US process}} }\over {L * eff * acc}} $

Where $  {acc = \frac{N_{selected}} { N_{tot WW}} }$,  $L$ is Luminosity, $eff$ is detector and selection effeciency, $N_{US process}$ is the unscaled number of events that pass selection cuts.


### Normalization uncertainty of a process is defined as:


$ \large{ {d_{process} * N_{process}}\over {L * eff * acc} }$

Where $d_{process}$ is the percent uncertainty on the theory / Monte Carlo for a given process.

### Relative uncertainty of a process us defined as:

Uncertainty divided by the cross section


In [3]:
total_number = 0
for flavor in results_0j.keys():
    for process in results_0j[flavor].keys():
        total_number += results_0j[flavor][process]
print total_number

NameError: name 'results_0j' is not defined

In [ ]:
summary_stat_unc = {}
summary_norm_unc = {}

relative_stat_unc = {}
relative_norm_unc = {}

post_cut_WW = results_0j["same_flavor"]["WW"] + results_0j["diff_flavor"]["WW"]

coeff = 1. / ( 19.3e3 * (3*.108)**2 *.15 * (post_cut_WW / (df[df.WW==1].shape[0]*scales["WW"])))


for flavor in results_0j.keys():
    summary_stat_unc[flavor] = {}
    summary_norm_unc[flavor] = {}
    
    relative_stat_unc[flavor] = {}
    relative_norm_unc[flavor] = {}
    
    for process in results_0j[flavor].keys():
        if "WW" not in process: 
            summary_stat_unc[flavor][process] = coeff * scales[process] * pow( results_0j[flavor][process] / scales[process], .5)  
            relative_stat_unc[flavor][process] = summary_stat_unc[flavor][process] / 58.77

            summary_norm_unc[flavor][process] = coeff * unc_mc_process[process] * results_0j[flavor][process]   
            relative_norm_unc[flavor][process] = summary_norm_unc[flavor][process] / 58.77


## Results 

In [ ]:
print "Stat uncertainty"
#create_table( summary_stat_unc, 2)

print "\n\nRelative Stat uncertainty"
#create_table( relative_stat_unc, 4)

In [ ]:
combine_unc(summary_stat_unc)

In [ ]:
print "Norm uncertainty"
create_table( summary_norm_unc, 2)


#print "\n\nRelative Norm uncertainty"
#create_table( relative_norm_unc, 4)

In [ ]:
combine_unc(summary_norm_unc)

In [ ]:
print "Stat + Norm uncert"
combine_unc({"stat":{'TT': 0.16, 'WZ': 0.23, 'ZZ': 0.08, 'DY': 0.62}, "norm": {'TT': 2.75, 'WZ': 0.43, 'ZZ': 0.07, 'DY': 0.44} })


In [ ]:
s_n = {'TT': 0.26, 'WZ': 0.3, 'ZZ': 0.08, 'DY': 0.15}#{'TT': 2.75, 'WZ': 0.43, 'ZZ': 0.07, 'DY': 0.44}
for i in s_n.keys():
    print i, s_n[i]/60.

In [ ]:
a = {'TT': 0.06, 'WZ': 0.22, 'ZZ': 0.09, 'DY': 0.39}#{'TT': 0.16, 'WZ': 0.23, 'ZZ': 0.08, 'DY': 0.62}
b=0
for i in a.keys():
   b+=a[i]**2
b**.5 / 60.

## Comparitive RF results

.98 analysis doesn't compare will with this result.  If I tighten the random forest cut to .9978 we get comparible yields.  However this selection is abit arbitary and not the point of this excersize. Never the less the results are below.

In [ ]:
rf_cut = (df.rf_result > .9988) & (df.numbExtraLep == 0)

In [ ]:
results_rf = {"same_flavor":{ process: df[ (df[process] == 1) & rf_cut & same_flavor_cut].shape[0]*scales[process] for process in scales.keys() if "WJ" not in process },
              "diff_flavor":{ process: df[ (df[process] == 1) & rf_cut & dif_flavor_cut].shape[0]*scales[process] for process in scales.keys() if "WJ" not in process }}

In [ ]:
create_table(results_rf, 0)

In [ ]:
summary_stat_unc_rf = {}
summary_norm_unc_rf = {}

relative_stat_unc_rf = {}
relative_norm_unc_rf = {}

post_cut_WW = results_rf["same_flavor"]["WW"] + results_rf["diff_flavor"]["WW"]

coeff = 1. / ( 19.3e3 * (3*.108)**2 *.15 * (post_cut_WW / (df[df.WW==1].shape[0]*scales["WW"])))


for flavor in results_rf.keys():
    summary_stat_unc_rf[flavor] = {}
    summary_norm_unc_rf[flavor] = {}
    
    relative_stat_unc_rf[flavor] = {}
    relative_norm_unc_rf[flavor] = {}
    
    for process in results_rf[flavor].keys():
        if "WW" not in process: 
            summary_stat_unc_rf[flavor][process] = coeff * scales[process] * pow( results_rf[flavor][process] / scales[process], .5)  
            relative_stat_unc_rf[flavor][process] = summary_stat_unc_rf[flavor][process] / 58.77

            summary_norm_unc_rf[flavor][process] = coeff * unc_mc_process[process] * results_rf[flavor][process]   
            relative_norm_unc_rf[flavor][process] = summary_norm_unc_rf[flavor][process] / 58.77

In [ ]:
print "Random forest stat uncertainty "
create_table(summary_stat_unc_rf, 3)
print "Random forest norm uncertainty "
create_table(summary_norm_unc_rf, 3)

In [ ]:
print "Basic cuts"
create_table(summary_stat_unc, 3)

In [ ]:
print "Random forest stat relative uncertainty "
create_table(relative_stat_unc_rf, 5)

In [ ]:
print "Basic cuts stat relative uncertainty "
create_table(relative_stat_unc, 5)

In [ ]:
combine_unc(summary_norm_unc_rf)
combine_unc(summary_stat_unc_rf)

In [ ]:
combine_unc({"norm":{'TT': 0.26, 'WZ': 0.3, 'ZZ': 0.08, 'DY': 0.15},"stat":{'TT': 0.06, 'WZ': 0.22, 'ZZ': 0.09, 'DY': 0.39}})

## Determining the best random forest cut value

This will be tackled the following file:

orig_cuts

In [ ]:
create_table(summary_norm_unc_rf, 3)

In [ ]:
#RF
same_flavor_stat={'TT': 0.028, 'WZ': 0.145, 'ZZ': 0.087, 'DY': 0.348}
diff_flavor_stat={'TT': 0.047, 'WZ': 0.171, 'ZZ': 0.032, 'DY': 0.172}
same_flavor_norm={'TT': 0.086, 'WZ': 0.174, 'ZZ': 0.078, 'DY': 0.145}
diff_flavor_norm={'TT': 0.242, 'WZ': 0.242, 'ZZ': 0.011, 'DY': 0.035}

In [ ]:
a=0
for process in same_flavor_stat.keys():
    a += same_flavor_stat[process]**2
    a += diff_flavor_stat[process]**2
    
    a += same_flavor_norm[process]**2
    a += diff_flavor_norm[process]**2

a**.5

In [ ]:
same_flavor_stat={'TT': 0.028, 'WZ': 0.145, 'ZZ': 0.087, 'DY': 0.348}
diff_flavor_stat={'TT': 0.047, 'WZ': 0.171, 'ZZ': 0.032, 'DY': 0.172}
same_flavor_norm={'TT': 0.086, 'WZ': 0.174, 'ZZ': 0.078, 'DY': 0.145}
diff_flavor_norm={'TT': 0.242, 'WZ': 0.242, 'ZZ': 0.011, 'DY': 0.035}